In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import strax

# ADC->PE conversions for XENON1T
to_pe = 1e-3 * np.array([7.05, 0.0, 0.0, 8.09, 4.38, 7.87, 3.58, 7.5, 7.44, 4.82, 7.07, 5.79,  0.0, 5.55, 7.95, 7.02, 6.39, 8.1, 7.15, 7.43, 7.15, 11.4, 3.97, 7.28,  5.41, 7.4, 0.0, 0.0, 7.04, 7.27, 4.22, 16.79, 4.14, 7.04, 0.0, 5.38,  7.39, 7.02, 4.53, 5.17, 7.13, 5.48, 4.6, 7.33, 6.14, 6.52, 7.59,  4.76, 7.56, 7.54, 4.57, 4.6, 7.12, 8.0, 4.7, 8.68, 3.74, 4.97, 10.36,  7.53, 6.02, 12.45, 0.0, 4.49, 4.82, 0.0, 8.13, 7.27, 3.55, 5.65,  4.55, 8.64, 7.97, 0.0, 3.57, 3.69, 5.87, 5.12, 9.8, 0.0, 5.08, 4.09,  3.87, 8.17, 6.73, 9.03, 0.0, 6.93, 0.0, 6.52, 7.39, 0.0, 4.92, 7.48,  5.82, 4.05, 3.9, 5.77, 8.14, 7.62, 7.61, 5.55, 0.0, 7.12, 5.02, 4.57,  4.46, 7.44, 3.57, 7.58, 7.16, 7.33, 7.69, 6.03, 5.87, 9.64, 4.68,  7.88, 0.0, 10.84, 7.0, 3.62, 7.5, 7.45, 7.69, 7.69, 3.49, 3.61, 7.44,  6.38, 0.0, 5.1, 3.72, 5.22, 0.0, 0.0, 4.43, 0.0, 3.87, 0.0, 3.6,  5.35, 8.4, 5.1, 6.45, 5.07, 4.28, 3.5, 0.0, 7.28, 0.0, 4.25, 0.0,  4.72, 6.26, 7.28, 5.34, 7.55, 3.85, 5.54, 7.5, 7.31, 0.0, 7.76, 7.57,  6.66, 7.29, 0.0, 7.59, 3.8, 3.58, 5.21, 4.29, 7.36, 7.76, 4.0, 6.23,  5.86, 0.0, 7.34, 3.58, 3.57, 5.26, 0.0, 7.67, 4.05, 4.3, 4.21, 7.59,  7.59, 0.0, 6.41, 4.86, 3.73, 5.09, 7.59, 7.64, 7.7, 0.0, 5.25, 8.0,  5.32, 7.91, 0.0, 4.41, 11.82, 0.0, 4.51, 7.05, 8.63, 5.12, 4.45,  4.03, 0.0, 0.0, 3.54, 4.18, 9.5, 3.64, 3.67, 7.28, 3.59, 5.03, 3.6,  5.4, 7.18, 3.73, 6.21, 6.47, 3.7, 7.69, 4.58, 7.46, 6.74, 0.0, 3.66,  7.49, 7.55, 3.64, 0.0, 7.34, 4.06, 3.74, 3.97, 0.0, 4.29, 4.96, 3.77,  8.57, 8.57, 8.57, 8.57, 8.57, 8.57, 214.29, 171.43, 171.43, 171.43,  171.43, 171.43])

In [ ]:
import numba

@numba.jit(nopython=True, nogil=True)
def find_split_points(w, min_height=0, min_ratio=0):
    """"Yield indices of significant local minima in w
    Plus, if there was at least one index, len(w)-1
    
    Peaks to the left and right both satisfy:
      - larger than minimum + min_height
      - larger than minimum * min_ratio
    """
    found_one = False
    last_max = 0
    min_since_max = 99999999999999.9
    min_since_max_i = 0

    for i, x in enumerate(w):
        if x < min_since_max:
            # New minimum since last max
            min_since_max = x
            min_since_max_i = i

        if min(last_max, x) > max(min_since_max + min_height,
                                  min_since_max * min_ratio):
            # Significant local minimum: tell caller, reset both max and min finder
            yield min_since_max_i
            found_one = True
            last_max = x
            min_since_max = 99999999999999.9
            min_since_max_i = i

        if x > last_max:
            # New max, reset minimum finder state
            # Notice this is AFTER the split check, to accomodate very fast rising second peaks
            last_max = x
            min_since_max = 99999999999999.9
            min_since_max_i = i
            
    if found_one:
        yield len(w) - 1

In [ ]:
records = strax.load('raw_data')
peaks = strax.load('processed_data')

In [ ]:
peak_i = 716 # 5733
p = peaks[peak_i]

def plot_peak(p, t0=None, **kwargs):
    n = p['length']
    if t0 is None:
        t0 = p['time']
    plt.plot((p['time'] - t0) + np.arange(n) * p['dt'], 
             p['data'][:n] / p['dt'], 
             linestyle='steps-mid',
             **kwargs)
    #for i in find_split_points(p['data'][:n], min_height=1000):
    #    plt.axvline(t0 + p['dt'] * i, c='k')
    plt.xlabel("Time (ns)")
    plt.ylabel("Sum waveform (PE / ns)")
    
plot_peak(peaks[5733])

In [ ]:
def split_peaks(peaks, records, to_pe, min_height=1000, min_ratio=0):
    is_split = np.zeros(len(peaks), dtype=np.bool_)
    
    new_peaks = _split_peaks(peaks, 
                             min_height=min_height, 
                             min_ratio=min_ratio, 
                             orig_dt=records[0]['dt'], 
                             is_split=is_split)
    strax.sum_waveform(new_peaks, records, to_pe)
    return strax.sort_by_time(np.concatenate([peaks[~is_split], 
                                              new_peaks]))

@utils.growing_result(dtype=peak_dtype(260), chunk_size=10)
@numba.jit(nopython=True)
def _split_peaks(new_peaks, peaks, min_height, min_ratio, orig_dt, is_split):
    offset = 0
    
    for p_i, p in enumerate(peaks):
        prev_split_i = 0
        
        for split_i in find_split_points(p['data'][:p['length']], 
                                         min_height=min_height, 
                                         min_ratio=min_ratio):
            is_split[p_i] = True
            
            r = new_peaks[offset]
            r['time'] = p['time'] + prev_split_i * p['dt']
            r['channel'] = p['channel']
            r['dt'] = orig_dt
            r['length'] = (split_i - prev_split_i) * p['dt'] / orig_dt
            
            offset += 1
            if offset == len(new_peaks):
                yield offset
                offset = 0
                
            prev_split_i = split_i

    yield offset

In [ ]:
%%time
new_peaks = split_peaks(peaks, records, to_pe)

In [ ]:
big_i = np.argmin(np.abs(new_peaks['time'] - peaks[5733]['time']))
for i in big_i + np.arange(0, 5):
    p = new_peaks[i]
    plot_peak(p,
              t0=new_peaks[big_i]['time'],
              label='%d: %d ns bins' % (i, p['dt']))
plt.legend(loc='best')

In [ ]:

strax.sum_waveform(new_peaks, records, to_pe)

In [ ]:
new_peaks[0]['length'] # peaks[716]['time']

In [ ]:
plt.plot(new_peaks[0]['data'])

In [ ]:
plt.plot(new_peaks[0]['data'])
plt.plot(new_peaks[1]['data'])
plt.plot(new_peaks[2]['data'])
plt.plot(new_peaks[3]['data'])